In [1]:
import tarfile
# open the tar file
with tarfile.open('/content/256_ObjectCategories.tar', 'r') as tar:
    # extract all files
    tar.extractall()

In [2]:
import os
import random
import shutil

# Set the number of images to use for training
num_train = 30

# Set the paths to the input dataset and the output folders
input_dir = '/content/256_ObjectCategories'
train_dir = '/content/256_ObjectCategories/train'
test_dir = '/content/256_ObjectCategories/test'

# Create the output folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Loop over each class in the input dataset
for class_name in os.listdir(input_dir):
    class_dir = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    # Create the class subdirectories in the output folders
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Get a list of all image files in the class directory
    image_files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]

    # Make sure there are enough images to use for training
    if len(image_files) < num_train:
        print(f'Skipping class {class_name}: not enough images ({len(image_files)})')
        continue

    # Randomly select num_train images for training, the rest for testing
    random.shuffle(image_files)
    train_files = image_files[:num_train]
    test_files = image_files[num_train:]

    # Copy the training images to the training folder
    for i in range(num_train):
        shutil.copy(train_files[i], train_class_dir)

    # Copy the testing images to the testing folder
    for i in range(num_train, len(image_files)):
        shutil.copy(test_files[i - num_train], test_class_dir)


Skipping class test: not enough images (0)
Skipping class train: not enough images (0)


In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/256_ObjectCategories/train'
test_dir = '/content/256_ObjectCategories/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 7710 images belonging to 259 classes.
Found 22897 images belonging to 259 classes.


In [26]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

# Define the data directories
train_dir = "/content/256_ObjectCategories/train"
test_dir = "/content/256_ObjectCategories/test"

# Define the image dimensions and batch size
img_width = 224
img_height = 224
batch_size = 32

# Define the number of classes in the dataset
num_classes = len(os.listdir(train_dir))

# Create an instance of the ImageDataGenerator class for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation datasets
train_dataset = train_datagen.flow_from_directory(train_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='input')

test_dataset = test_datagen.flow_from_directory(test_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode='input')

# Define the autoencoder model architecture
inputs = keras.Input(shape=(img_width, img_height, 3))
x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(16, 3, activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D(2, padding='same')(x)

x = layers.Conv2D(16, 3, activation='relu', padding='same')(encoded)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
decoded = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

model = keras.Model(inputs=inputs, outputs=decoded)

# Compile the model
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

# Train the model with data augmentation and autoencoder feature extraction
start_time = time.time()

history = model.fit(train_dataset,
                    steps_per_epoch=train_dataset.samples // batch_size,
                    epochs=10,
                    validation_data=test_dataset,
                    validation_steps=test_dataset.samples // batch_size)

end_time = time.time()

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print('Test accuracy:', test_acc)


Found 7710 images belonging to 259 classes.
Found 22897 images belonging to 259 classes.
Epoch 1/10
240/240 [==============================] - 235s 860ms/step - loss: 0.0160 - accuracy: 0.6018 - val_loss: 0.0080 - val_accuracy: 0.7649
Epoch 2/10
240/240 [==============================] - 254s 1s/step - loss: 0.0050 - accuracy: 0.7666 - val_loss: 0.0066 - val_accuracy: 0.7128
Epoch 3/10
240/240 [==============================] - 204s 853ms/step - loss: 0.0041 - accuracy: 0.7384 - val_loss: 0.0060 - val_accuracy: 0.7937
Epoch 4/10
240/240 [==============================] - 205s 855ms/step - loss: 0.0036 - accuracy: 0.7251 - val_loss: 0.0056 - val_accuracy: 0.7030
Epoch 5/10
240/240 [==============================] - 252s 1s/step - loss: 0.0034 - accuracy: 0.7168 - val_loss: 0.0054 - val_accuracy: 0.7022
Epoch 6/10
240/240 [==============================] - 253s 1s/step - loss: 0.0032 - accuracy: 0.7270 - val_loss: 0.0052 - val_accuracy: 0.7207
Epoch 7/10
240/240 [========================

In [27]:
start_time_2 = time.time()

history_2 = model.fit(train_dataset,
                    steps_per_epoch=train_dataset.samples // batch_size,
                    epochs=10,
                    validation_data=test_dataset,
                    validation_steps=test_dataset.samples // batch_size)

end_time_2 = time.time()

# Evaluate the model on the test dataset
test_loss_2, test_acc_2 = model.evaluate(test_dataset, verbose=2)
print('Test accuracy:', test_acc_2)


Epoch 1/10
240/240 [==============================] - 200s 835ms/step - loss: 0.0027 - accuracy: 0.7696 - val_loss: 0.0047 - val_accuracy: 0.7398
Epoch 2/10
240/240 [==============================] - 251s 1s/step - loss: 0.0028 - accuracy: 0.7733 - val_loss: 0.0050 - val_accuracy: 0.6632
Epoch 3/10
240/240 [==============================] - 252s 1s/step - loss: 0.0026 - accuracy: 0.7752 - val_loss: 0.0046 - val_accuracy: 0.7488
Epoch 4/10
240/240 [==============================] - 252s 1s/step - loss: 0.0025 - accuracy: 0.7828 - val_loss: 0.0045 - val_accuracy: 0.7662
Epoch 5/10
240/240 [==============================] - 201s 837ms/step - loss: 0.0026 - accuracy: 0.7698 - val_loss: 0.0045 - val_accuracy: 0.7455
Epoch 6/10
240/240 [==============================] - 201s 837ms/step - loss: 0.0025 - accuracy: 0.7835 - val_loss: 0.0052 - val_accuracy: 0.7152
Epoch 7/10
240/240 [==============================] - 254s 1s/step - loss: 0.0025 - accuracy: 0.7766 - val_loss: 0.0044 - val_accurac

In [28]:

# Calculate the average testing accuracy and training time across the two runs
avg_test_acc = (test_acc + test_acc_2) / 2
total_training_time = end_time_2 - start_time + (end_time - start_time_2)

print('Average test accuracy:', avg_test_acc)
print('Total training time (seconds):', total_training_time)

Average test accuracy: 0.7791490852832794
Total training time (seconds): 4674.177948951721
